In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
#image dimensions
img_width, img_height = 200, 200

In [3]:
train_data_dir = './data/train'
test_data_dir  = './data/test'
num_train_samples = 59850 #29925 in both classes  
num_test_samples  = 22828 #7498 in black, 15330 in death
epochs = 20 
batch_size = 32

In [12]:
if K.image_data_format() == 'channel_first':
    input_shape = (3, img_width, img_height)   #change 3 to 1 for grayscale
else:
    input_shape = (img_width, img_height, 3)

In [13]:

model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=num_train_samples//batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=num_test_samples//batch_size)

#roughly % accuracy on rgb
#roughly 67% accuracy on greyscale

Found 59849 images belonging to 2 classes.
Found 22828 images belonging to 2 classes.
Epoch 1/20
1870/1870 [==============================] - 6981s 4s/step - loss: 0.6520 - acc: 0.6437 - val_loss: 0.6555 - val_acc: 0.6737
Epoch 2/20
1870/1870 [==============================] - 6225s 3s/step - loss: 0.6322 - acc: 0.6627 - val_loss: 0.7421 - val_acc: 0.6267
Epoch 3/20
1870/1870 [==============================] - 6182s 3s/step - loss: 0.6324 - acc: 0.6645 - val_loss: 0.5867 - val_acc: 0.7052
Epoch 4/20
1870/1870 [==============================] - 5999s 3s/step - loss: 0.6294 - acc: 0.6641 - val_loss: 0.5737 - val_acc: 0.7142
Epoch 5/20
1870/1870 [==============================] - 26596s 14s/step - loss: 0.6264 - acc: 0.6661 - val_loss: 0.6302 - val_acc: 0.6781
Epoch 6/20
1870/1870 [==============================] - 6080s 3s/step - loss: 0.6247 - acc: 0.6676 - val_loss: 0.6767 - val_acc: 0.6358
Epoch 7/20
1870/1870 [==============================] - 5914s 3s/step - loss: 0.6233 - acc: 0.66

In [16]:
model.save_weights('rgb_model.h5')